# **Classification model for detection of Fake Image**
# (AI image & Adversarial Attacked Image)

Team:

*   Anello Fabrizio
*   Benestare Domenico
*   Margheriti Vincenzo

***Big Data Analytics***, corso magistrale di Ingegneria Gestionale Politecnico di Bari, Professore ***Yashar Deldjoo***.

L'obbiettivo del progetto è quello di creare un classificatore attraverso tensorflow e le reti neurali che riesca a classificare un'immagine reale e un'immagine fake. In particolare si confronteranno due tipi di immagini fake:


*   quelle generate dall'intelligenza artificiale, ad esempio attraverso strumenti come Stable Diffusion
*   quelle generate attraverso attacchi adversariali, ovvero un rumore invisibile agli occhi umani che crea disturbi ai modelli di classificazione

# Detection of AI Image

Si inizia con le immagini generate da AI. Si è utilizzato un sottoinsieme di circa 14 000 immagini del Database CISFAKE (fonte: Kaggle), che presenta un totale di circa 180 000 immagini. Il database parte dal famoso database di immagini cifar10 e aggiunge delle copie delle stesse immagini generate attraverso Stable Diffusion.

![CIFAKE](https://storage.googleapis.com/kaggle-datasets-images/3041726/5227744/64b8381e45aef2060808e31584ed141f/dataset-cover.png?t=2023-03-24-13-29-07)

Si importeranno manualmente le immagini, divise in cartelle, e caricate sul modello attraverso google drive. Un set di circa 10 000 immagini (5 000 reali e 5 000 generate AI) sarà usato per il training, mentre circa 4 000 immagini per il test.



**keras.preprocessing.image.ImageDataGenerator**: Questa classe fornisce metodi per generare e pre-elaborare immagini per l'addestramento dei modelli neurali. Viene utilizzata per creare generatori di dati per caricare le immagini dai set di dati.

**keras.models.Sequential**: Questa classe consente di creare modelli di reti neurali sequenziali. Viene utilizzata per definire il modello di rete neurale.

**keras.layers**: Questo modulo fornisce vari strati (o livelli) che possono essere aggiunti al modello. Alcuni di questi strati, come Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, e Activation, sono utilizzati per costruire il modello di rete neurale.

**keras.optimizers.Adam**: Questa classe implementa l'ottimizzatore Adam, un algoritmo di ottimizzazione ampiamente utilizzato per addestrare reti neurali. Viene utilizzato durante la compilazione del modello.

**matplotlib.pyplot**: Questa libreria consente di visualizzare le immagini e i grafici. Viene utilizzata per visualizzare le immagini di esempio e eventuali grafici di addestramento o valutazione del modello.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.model_selection import train_test_split

Dopodichè viene montato Google Drive su Google Colab e vengono definiti i percorsi per le cartelle di training e test del dataset.






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Definizione dei percorsi
database_path = "/content/drive/MyDrive/ciskfake_big"

train_dir = os.path.join(database_path, 'train')
test_dir = os.path.join(database_path, 'test')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



Nel seguente passaggio, vengono definiti i generatori di dati utilizzando la classe ImageDataGenerator di Keras. Questi generatori verranno utilizzati per caricare e pre-elaborare le immagini dai set di dati di addestramento e test:

**train_datagen = ImageDataGenerator(rescale=1./255)**: Viene istanziato un oggetto ImageDataGenerator per il set di dati di addestramento. Il parametro rescale=1./255 normalizza i valori dei pixel delle immagini, dividendo ciascun valore per 255 e quindi scalando i valori dei pixel nell'intervallo [0, 1].
.

**train_generator = train_datagen.flow_from_directory(...)**: Il metodo flow_from_directory dell'oggetto train_datagen crea un generatore di dati per il set di addestramento. Questo metodo carica le immagini dalla directory specificata (train_dir), le ridimensiona alle dimensioni specificate (target_size=(32, 32)), le divide in batch di dimensione 32 (batch_size=32), e le etichetta in base alla struttura delle sottocartelle. Poiché è un problema di classificazione binaria, viene impostato class_mode='binary'. Inoltre, specificando classes=['REAL', 'FAKE'], si indicano le classi delle immagini come "REAL" e "FAKE".

La stessa cosa viene poi effettuata per i dati di test.

In [ ]:
# Definizione dei generatori di dati
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='binary',
    classes=['REAL', 'FAKE'])

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='binary',
    classes=['REAL', 'FAKE'])

Found 10010 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


Viene poi definito il modello di rete neurale utilizzando il framework Keras:

**model = Sequential([...])**: Viene istanziato un modello sequenziale, che rappresenta una sequenza lineare di livelli. Questo tipo di modello è adatto per la definizione di modelli di rete neurale feedforward, in cui l'output di un livello diventa l'input del successivo.

**Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3))**: Viene aggiunto un livello di convoluzione 2D con 64 filtri, una dimensione del kernel di (3, 3), e attivazione ReLU. Questo è il primo livello del modello e ha una forma di input specificata come (32, 32, 3), che rappresenta l'altezza, la larghezza e i canali delle immagini di input.

**MaxPooling2D()**: Viene aggiunto un livello di pooling massimo 2D. Questo livello riduce la dimensione spaziale dell'output dei livelli precedenti, riducendo il numero di parametri e il costo computazionale della rete.

**Conv2D(64, (3, 3), activation='relu')**: Viene aggiunto un secondo livello di convoluzione 2D con 64 filtri e attivazione ReLU. Questo livello estrae ulteriori caratteristiche dalle immagini di input.

**MaxPooling2D(**): Viene aggiunto un altro livello di pooling massimo 2D per ridurre ulteriormente la dimensione spaziale dell'output.

**Flatten()**: Viene aggiunto un livello di appiattimento che converte l'output dei livelli convoluzionali in un vettore monodimensionale, preparandolo per l'input ai livelli densamente connessi.

**Dense(64)**: Viene aggiunto un livello denso con 64 unità nascoste. Questo livello è completamente connesso al livello precedente e applica una trasformazione lineare seguita da un'attivazione ReLU.

**BatchNormalization()**: Viene aggiunto un livello di normalizzazione batch, che normalizza gli output del livello precedente in modo da migliorare la stabilità e l'efficienza della rete neurale.

**Activation('relu')**: Viene aggiunto un livello di attivazione ReLU per introdurre non linearità nell'output del livello denso.

**Dense(1, activation='sigmoid')**: Viene aggiunto un ultimo livello denso con 1 unità nascosta e attivazione sigmoid. Questo livello produce l'output finale della rete, che rappresenta la probabilità che l'input appartenga a una delle due classi (REAL o FAKE) in un problema di classificazione binaria. La funzione di attivazione sigmoid è utilizzata poiché restituisce valori nell'intervallo [0, 1], che possono essere interpretati come probabilità.

In [ ]:
# Definizione del modello
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(64),
    BatchNormalization(),
    Activation('relu'),
    Dense(1, activation='sigmoid')
])

Vengono eseguite le successive operazioni:

***1. Compilazione del modello***: Il modello viene compilato con il metodo compile(), dove vengono specificati l'ottimizzatore, la funzione di perdita e le metriche da monitorare durante l'addestramento. In particolare:

**optimizer='adam'**: Viene utilizzato l'ottimizzatore Adam, un metodo di ottimizzazione basato su stime adattive dei momenti del gradiente, che non rimane con un learning rate fisso.
**loss='binary_crossentropy'**: Viene utilizzata la funzione di perdita binary_crossentropy, adatta per problemi di classificazione binaria.
**metrics=['accuracy']**: La metrica di valutazione dell'accuratezza viene monitorata durante l'addestramento.

***2.Addestramento del modello***: Il modello viene addestrato utilizzando il metodo fit(), che richiede il generatore di dati per il set di addestramento (train_generator) e il generatore di dati per il set di test (test_generator). Altri parametri includono:
**steps_per_epoch=train_generator.samples // 32**: Il numero di passaggi per epoch, calcolato come il numero totale di campioni di addestramento diviso per il batch size.
**epochs=5**: Il numero di epochs di addestramento.
**validation_data=test_generator**: Il generatore di dati per il set di test viene utilizzato per la validazione durante l'addestramento.
**validation_steps=test_generator.samples // 32**: Il numero di passaggi per epoch durante la validazione, calcolato come il numero totale di campioni di test diviso per il batch size.

Questo blocco di codice addestra effettivamente il modello utilizzando i dati di addestramento e ne valuta le prestazioni utilizzando i dati di test.

In [ ]:
# Compilazione del modello
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Addestramento del modello
history = model.fit(
    train_generator,
    #steps_per_epoch=train_generator.samples // 32,
    epochs=5,
    validation_data=test_generator
    #validation_steps=test_generator.samples // 32
)

Epoch 1/5
313/313 [==============================] - 4459s 14s/step - loss: 0.3573 - accuracy: 0.8422 - val_loss: 1.1524 - val_accuracy: 0.5760
Epoch 2/5
313/313 [==============================] - 59s 188ms/step - loss: 0.2926 - accuracy: 0.8801 - val_loss: 0.3673 - val_accuracy: 0.8393
Epoch 3/5
313/313 [==============================] - 59s 188ms/step - loss: 0.2570 - accuracy: 0.8934 - val_loss: 1.0237 - val_accuracy: 0.6690
Epoch 4/5
313/313 [==============================] - 51s 162ms/step - loss: 0.2208 - accuracy: 0.9089 - val_loss: 0.2655 - val_accuracy: 0.8850
Epoch 5/5
313/313 [==============================] - 59s 187ms/step - loss: 0.1966 - accuracy: 0.9223 - val_loss: 0.3667 - val_accuracy: 0.8550


Viene infine eseguita la valutazione del modello utilizzando il metodo evaluate() su un generatore di dati di test. Questo metodo calcola la perdita e le metriche specificate durante la compilazione del modello utilizzando i dati di test.

Valutazione del modello: Viene chiamato il metodo evaluate() del modello passando il generatore di dati di test test_generator. Questo metodo restituisce la perdita e le metriche calcolate utilizzando i dati di test.

Assegnazione delle variabili: I valori di perdita e accuratezza restituiti da evaluate() vengono assegnati alle variabili test_loss e test_acc, rispettivamente.

Stampa dei risultati: Infine, i risultati della valutazione vengono stampati utilizzando la funzione print(). In particolare, viene stampata l'accuratezza del modello sul set di test.

Questo blocco di codice fornisce un modo per valutare le prestazioni del modello dopo l'addestramento utilizzando i dati di test. La misurazione dell'accuratezza sul set di test fornisce un'indicazione dell'efficacia del modello nella generalizzazione a nuovi dati non visti durante l'addestramento.

In [ ]:
# Valutazione del modello
test_loss, test_acc = model.evaluate(test_generator)
print('Test Accuracy:', test_acc)

125/125 [==============================] - 13s 108ms/step - loss: 0.3667 - accuracy: 0.8550
Test Accuracy: 0.8550000190734863


# Adversarial Attacked Image

Lo stesso modello è applicato a un database di immagini generate con attacchi adversariali. Gli attacchi adversariali creano rumore ai pixel delle immagini originali, un rumore invisibile agli occhi umani ma visibile agli occhi dei modelli di classificazione, che quindi sbagliano. Si cerca di capire, rispetto al tipo di classificazione precedente, con lo stesso modello, qual è la pericolosità di tali immagini.

Il database usato è stato generato da noi grazie a librerie di adversarial attack, di seguito lo script della generazione:
[Google Collab Generazione Database](https://colab.research.google.com/drive/168hfC9qkH9JvgUXUEtLhpX50feQDSzqH?usp=sharing)

Si è partiti anche qui da un sottoinsieme del database cifar10, e le stesse immagini sono state poi attaccate, arrivando ad ottenere delle coppie di immagini visibilmente uguali all'occhio mano, ma in realtà diverse.

![Adversarial Attack](https://miro.medium.com/v2/resize:fit:1400/1*PmCgcjO3sr3CPPaCpy5Fgw.png)

Create le immagini, sono state scaricate in formato .npz (numpy) e vengono caricate sul seguente codice attraverso google drive.


In [ ]:
# Caricamento dei dati da Google Drive
drive_path = '/content/drive/My Drive/'

# Carica il file attacked_dataset.npz
attacked_data = np.load(drive_path + 'attackeddatabase_big.npz')
original_data = np.load(drive_path + 'originaldatabase_big.npz')

Il database generato è composto da 14 000 immagini (7 000 originali e 7000 attaccate). Viene poi splittato in test e training data e normalizzato.



In [ ]:
# Estrazione delle immagini e delle etichette
attacked_images, attacked_labels = attacked_data['images'], np.ones(attacked_data['images'].shape[0])
original_images, original_labels = original_data['images'], np.zeros(original_data['images'].shape[0])

# Concatenazione dei dati
images = np.concatenate([original_images, attacked_images], axis=0)
labels = np.concatenate([original_labels, attacked_labels], axis=0)

# Divisione dei dati in training e test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalizzazione delle immagini
X_train = X_train / 255.0
X_test = X_test / 255.0

Attraverso lo stesso modello tensorflow già definito, si può procedere al fit.

In [ ]:
# Addestramento del modello
history = model.fit(
    X_train, y_train,
    epochs=5,
    validation_data=(X_test, y_test)
)

Epoch 1/5
700/700 [==============================] - 32s 46ms/step - loss: 0.6956 - accuracy: 0.5005 - val_loss: 1.4773 - val_accuracy: 0.5021
Epoch 2/5
700/700 [==============================] - 33s 47ms/step - loss: 0.6930 - accuracy: 0.5046 - val_loss: 0.8903 - val_accuracy: 0.5021
Epoch 3/5
700/700 [==============================] - 38s 54ms/step - loss: 0.6927 - accuracy: 0.5118 - val_loss: 0.9739 - val_accuracy: 0.5021
Epoch 4/5
700/700 [==============================] - 37s 53ms/step - loss: 0.6926 - accuracy: 0.5169 - val_loss: 3.6434 - val_accuracy: 0.5021
Epoch 5/5
700/700 [==============================] - 39s 55ms/step - loss: 0.6926 - accuracy: 0.5164 - val_loss: 1.9866 - val_accuracy: 0.4979


In [ ]:
# Valutazione del modello
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)

175/175 [==============================] - 2s 12ms/step - loss: 1.9866 - accuracy: 0.4979
Test Accuracy: 0.49785715341567993


Si può notare come l'accurattezza sia molto bassa, pessima rispetto allo stesso modello di classificazione applicato alle immagini generate da AI. Si può dunque affermare che rispetto alle immagini generate da AI, facilmente classificabili dalle reti neurali, le immagini attaccate avversalmente rappresentano un vero pericolo, che potrebbe compromettere sistemi di sicurezza, di robotica, di guida autonoma, di privacy, ecc.

Inoltre, si è osservato come al crescere del database, cresce l'accuratezza del modello applicato sulle immagini AI (si è partiti con un'accuratezza di circa il 70% su 1000 dati fino ad arrivare all'85% su 14000 dati. Sicuramente utilizzando l'intero database, di circa 180 000 immagini, l'accuratezza sarebbe prossima al 100%, ma purtroppo tale numero di immagini rappresenta un limite su google collab). Mentre l'accuratezza del modello applicato sulle immagini attaccate rimane invariato (è addirittura diminuita rispetto a un database precedente più piccolo).
